In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
# C:\Users\Dell\DataScience\Lums\CS5310\MiniProject\Task3
load_dir = "C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task3/"

cams_detail = {
    "Camera-1": load_dir + "Videos/campus4-c0.mp4",
    "Camera-2": load_dir + "Videos/campus4-c1.mp4",
    "Camera-3": load_dir + "Videos/campus4-c2.mp4"    
}

In [3]:
def getVideoFrame(cams_detail):
    cams_frame = {}
    for key, video_address in cams_detail.items():        
        
        cam = cv2.VideoCapture(video_address)
        ret, frame = cam.read()
        if ret == True:
            cams_frame[key] = frame
        cam.release()
        
    return cams_frame

cams_frame = getVideoFrame(cams_detail)

In [4]:
# save single image from each camera in Images folder
for key, frame in cams_frame.items():
    
    save_dir = load_dir + 'Images' + f'/{key}.jpg'
    ret = cv2.imwrite(save_dir, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    if ret == True:
        print(key,"Image Saved: ",frame.shape, save_dir)
        cv2.imshow(key, frame)
        cv2.waitKey(0)

cv2.destroyAllWindows()

Camera-1 Image Saved:  (288, 360, 3) C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task3/Images/Camera-1.jpg
Camera-2 Image Saved:  (288, 360, 3) C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task3/Images/Camera-2.jpg
Camera-3 Image Saved:  (288, 360, 3) C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task3/Images/Camera-3.jpg


### Select Points

In [5]:
imagePath = {
    "Camera-1": load_dir + "/Images/Camera-1.jpg",
    "Camera-2": load_dir + "/Images/Camera-2.jpg",
    "Camera-3": load_dir + "/Images/Camera-3.jpg",
    "Top-View": load_dir + "/Images/top_view.jpg"
}

In [6]:
def click_event(event, x, y, flags, params): 
    if event == cv2.EVENT_LBUTTONDOWN: 
        print(params[0], [x, y]) 
        params[2][params[0]].append([x,y])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.circle(params[1], (x,y), 2, (22, 35,255 ), -1)
        cv2.imshow(params[0], params[1])

In [7]:
def mark_points(image_name, path_image, top_image_name, path_top_image):

    points = defaultdict(list)
    
    image = cv2.imread(path_image, 1)
    top_image = cv2.imread(path_top_image, 1)

    cv2.imshow(image_name, image)
    cv2.setMouseCallback(image_name, click_event, (image_name, image, points)) 

    cv2.imshow(top_image_name, top_image)
    cv2.setMouseCallback(top_image_name, click_event, (top_image_name,top_image, points)) 
    
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 
    return points, image, top_image

In [8]:
def show_images_to_mark(image_name, top_image_name):
    path_image = imagePath[image_name]
    path_top_image = imagePath[top_image_name]
    
    points, image, top_image  = mark_points(image_name, path_image, top_image_name, path_top_image)
    
    # path of images marked is Images/Marked
    cv2.imwrite(load_dir + f'/Images/Marked/{image_name}.jpg' , image, [cv2.IMWRITE_JPEG_QUALITY, 100])
    cv2.imwrite(load_dir + f'/Images/Marked/{image_name + "-" + top_image_name}.jpg' , top_image, [cv2.IMWRITE_JPEG_QUALITY, 100])
    
    print("Selected Points:")
    print(image_name, points[image_name])
    print(top_image_name, points[top_image_name])
    
    return np.array(points[image_name]), np.array(points[top_image_name])

## Calculate Homography 

In [9]:
def calculate_homography(image_points, top_view_points):    
    if len(image_points) == len(top_view_points) & len(image_points) > 3 :
        Homography_matrix[image_name] = cv2.findHomography(image_points,top_view_points)[0]
        print(Homography_matrix[image_name])
        
        save_at = f"{load_dir}/Images/Homography/H-{image_name}.csv"
        np.savetxt(save_at, Homography_matrix[image_name], delimiter=',')
        print("Saved at", save_at)
        
    else:
        print(f"Can't apply on image_points: { len(image_points)},  top_view_points: {len(top_view_points)} ")

## Apply Homograhy 

In [10]:
def warpImages(img1, img2, H):
    h1,w1 = img1.shape[:2]
    h2,w2 = img2.shape[:2]
    pts1 = np.float32([[0,0],[0,h1],[w1,h1],[w1,0]]).reshape(-1,1,2)
    pts2 = np.float32([[0,0],[0,h2],[w2,h2],[w2,0]]).reshape(-1,1,2)
    pts2_ = cv2.perspectiveTransform(pts2, H)
    pts = np.concatenate((pts1, pts2_), axis=0)
    [xmin, ymin] = np.int32(pts.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(pts.max(axis=0).ravel() + 0.5)
    t = [-xmin,-ymin]
    Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]])
    
    result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))
    return result

In [11]:
def apply_homography(image, top_image, homography):
    result = warpImages(top_image, image, homography)
    cv2.imshow(image_name, image)
    cv2.imshow(image_name + "-Out",result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    
    return result

## Run 

In [12]:
# path of images marked is Images/Marked
def homography(image_name, top_image_name):

    image_points, top_view_points =  show_images_to_mark(image_name, top_image_name)

    calculate_homography(image_points, top_view_points)

    image = cv2.imread(imagePath[image_name], 1)
    top_image = cv2.imread(imagePath[top_image_name], 1)
    
    homography = Homography_matrix[image_name]

    result_image = apply_homography(image, top_image, homography)

Uncomment below code to mark homography

In [13]:
# Homography_matrix = {}

# source_images = ["Camera-1", "Camera-2", "Camera-3"]
# # source_images = ["Camera-3"]

# target_image = "Top-View"

# for image_name in source_images:
#     homography(image_name, target_image)

## Homography on Video

In [14]:
source_images = ["Camera-1", "Camera-2", "Camera-3"]

Homography_matrix = {}
Homography_matrix["Camera-1"] = np.array([[-0.211332, -0.405226, 70.781223], [-0.019746, -1.564936, 226.377280], [-0.000025, -0.001961, 0.160791]])
Homography_matrix["Camera-2"] = np.array([[0.000745, 0.350335, -98.376103], [-0.164871, -0.390422, 54.081423], [0.000021, -0.001668, 0.111075]])
Homography_matrix["Camera-3"] = np.array([[0.089976, 1.066795, -152.055667], [-0.116343, 0.861342, -75.122116], [0.000015, 0.001442, -0.064065]])

In [15]:
cameras = {
    "Camera-1": load_dir + "Videos/campus4-c0.mp4",
    "Camera-2": load_dir + "Videos/campus4-c1.mp4",
    "Camera-3": load_dir + "Videos/campus4-c2.mp4"    
}

In [16]:
def Get_Available_Cams(cameras):
    available_cams = {}

    for key, video_address in cameras.items():
        cam = cv2.VideoCapture(video_address)
        if cam.isOpened() == True:
            available_cams[key] = cam
            
    return available_cams

In [17]:
def Apply_Homograpy_On_Frames(current_frame):
        
    merge_image = []
    flag = True
    
    for key in list(current_frame.keys()):
        new_image = cv2.warpPerspective(current_frame[key], Homography_matrix[key] ,(700,700)).astype('uint32')
        if flag == True:
            merge_image = new_image
            flag = False
        else:
            merge_image += new_image
        
    
    length = len(current_frame.keys())
    merge_image =  np.array((np.array(merge_image) / length)).astype('uint8')
    
    cv2.imshow("Merged", merge_image)
    cv2.waitKey(1)
    
    return merge_image

In [18]:
def Read_Image(available_cams):
    
    cam_keys = list(available_cams.keys())
    save_path =  load_dir + 'FinalVideo/top_view_video.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    top_view_video = cv2.VideoWriter(save_path, fourcc, 25, (700,700))
    
    while len(cam_keys) == len(available_cams):
        current_frame = {}
        for i , key in enumerate(cam_keys):            
            is_open , frame = available_cams[key].read()
            if is_open == True:
                current_frame[key] = frame
            else:
                cam_keys.pop(i)
        
        if len(cam_keys) == len(available_cams):
            merge_image = Apply_Homograpy_On_Frames(current_frame)
            top_view_video.write(merge_image)
    
    for key in available_cams.keys():
        available_cams[key].release()

    top_view_video.release()

In [19]:
top_view_path = load_dir + "Images/top_view.jpg"
top_image = cv2.imread(top_view_path, 1)

camerasAvailable = Get_Available_Cams(cameras)
Read_Image(camerasAvailable)
cv2.destroyAllWindows()